# U-Net Model
在本专题实践课程的作业中，你将使用一种称为“U-Net”的网络架构。这种网络架构的名称来源于它的U形形状，如下图所示（图片来U-net原文）：

<img src="images/unet.png" alt="U-net Image" width="600"/>

U网络通常用于图像分割，是本专题实践课程作业的任务。在开始作业之前，我们先来熟悉这个架构。

如图所示，该架构具有一系列由最大池操作连接的下行-卷积，然后是一系列由上采样和级联操作连接的上行-卷积。每个下行-卷积也直接连接到网络上采样部分中的级联操作。有关U-Net架构的更多详细信息，请查看原始论文[U-Net paper by Ronneberger et al. 2015](https://arxiv.org/abs/1505.04597)。

这里，我们将通过Keras创建一个标准的U-Net网络。

In [6]:
# Import the elements you'll need to build your U-Net
import keras
from keras import backend as K
from keras.models import Model
from keras.layers import Input, Conv3D, MaxPooling3D, UpSampling3D, Activation, BatchNormalization, PReLU, Conv3DTranspose
from keras.optimizers import adam_v2
from keras.layers import concatenate
# Set the image shape to have the channels in the first dimension
K.set_image_data_format("channels_first")

import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

## 1. U-Net的"深度" 

U-Net的“深度”等于将使用的下行-卷积的数量。在上图中，深度为4，因为有4个下行-卷积沿着左侧运行，包括U型结构的底部。

在本实践中，你将使用U-Net的深度为2，即网络中将有2个下行-卷积。


### 1.1 输入层和它的"Depth"

在本实践中，你将进行3D图像分割，也就是说，除了“height”和“width”，你的输入层还将具有“length”。我们有意使用“length”一词而不是“depth”来描述输入的第三个空间维度，以免将其与上述定义的网络深度混淆。

输入层的形状为`(num_channels, height, width, length)`，其中`num_channels`可以想象成图像中的颜色通道，`height`、`width`和`length`是输入的大小。

对于本任务，参数设置为：
- num_channels: 4
- height: 160
- width: 160
- length: 16

In [7]:
# Define an input layer tensor of the shape you'll use in the assignment
input_layer = Input(shape=(4, 160, 160, 16))
input_layer

<KerasTensor: shape=(None, 4, 160, 160, 16) dtype=float32 (created by layer 'input_1')>

注意：张量形状的第一个维度是批量大小。所以张量的维数是：（batch_size，num_channels，height，width，length）

## 2. 编码器 (下行) 
这里，将从构建网络中的下行路径（U-Net的左侧）开始。沿着这条路径向下移动时，输入的`(height, width, length)`会变小，通道数也会增加。

### 2.1 Depth 0

这里的"depth 0"指的是U-net中第一次下行-卷积的深度。

为每个深度和该深度内的每个层指定filters的数量。

计算filters数量的公式为:
$$filters_{i} = 32 \times (2^{i})$$

其中 $i$ 是当前网络深度。

所以在深度 $i=0$:
$$filters_{0} = 32 \times (2^{0}) = 32$$

### 2.2 Layer 0
每个深度都有两个卷积层。

运行下一个单元格以创建第一个3D卷积。

In [8]:
# Define a Conv3D tensor with 32 filters
down_depth_0_layer_0 = Conv3D(filters=32, 
                              kernel_size=(3,3,3),
                              padding='same',
                              strides=(1,1,1)
                              )(input_layer)
down_depth_0_layer_0

<KerasTensor: shape=(None, 32, 160, 160, 16) dtype=float32 (created by layer 'conv3d')>

注意，对于32个filters，上面得到的结果是具有32个通道的张量。

运行下一个单元格，将relu激活添加到第一个卷积层。

In [9]:
# Add a relu activation to layer 0 of depth 0
down_depth_0_layer_0 = Activation('relu')(down_depth_0_layer_0)
down_depth_0_layer_0

<KerasTensor: shape=(None, 32, 160, 160, 16) dtype=float32 (created by layer 'activation')>

### 2.3 Depth 0, Layer 1
对于depth 0的layer 1，计算filters数量的公式为：
$$filters_{i} = 32 \times (2^{i}) \times 2$$

其中 $i$ 是当前网络深度。 
- 注意： 公式最后的'$\times~2$' 不用于layer 0.


所以layer 1中的depth $i=0$ ：
$$filters_{0} = 32 \times (2^{0}) \times 2 = 64$$

In [5]:
# Create a Conv3D layer with 64 filters and add relu activation
down_depth_0_layer_1 = Conv3D(filters=64, 
                kernel_size=(3,3,3),
                padding='same',
                strides=(1,1,1)
               )(down_depth_0_layer_0)
down_depth_0_layer_1 = Activation('relu')(down_depth_0_layer_1)
down_depth_0_layer_1

<KerasTensor: shape=(None, 64, 160, 160, 16) dtype=float32 (created by layer 'activation_1')>

### 2.4 Max pooling
在U-Net架构中，在每个下行卷积之后都有一个最大池化操作（不包括U-Net底部的最后一个下行卷积）。通常，这意味着每次下行卷积之后会添加最大池化操作，直到（但不包括）`depth - 1`的下行卷积（因为从0开始计数）。

对于本实践:
- 所构建的U-Net网络的总深度是2
- 所以U-Net底部的深度指数为: $2-1 = 1$.
- 到目前为止，我们只定义了$depth=0$下行卷积，因此下一步要做的是添加最大池化操作


运行下一个单元格，将最大池化操作添加到U-Net。

In [6]:
# Define a max pooling layer
down_depth_0_layer_pool = MaxPooling3D(pool_size=(2,2,2))(down_depth_0_layer_1)
down_depth_0_layer_pool

<KerasTensor: shape=(None, 64, 80, 80, 8) dtype=float32 (created by layer 'max_pooling3d')>

#### 2.4.1 Depth 1, Layer 0

在depth 1, layer 0, 用于计算filters数量的公式为:
$$filters_{i} = 32 \times (2^{i})$$

其中 $i$ 是当前网络深度。

所以在 depth $i=1$:
$$filters_{1} = 32 \times (2^{1}) = 64$$

运行下一个单元格，将带有relu激活层的3D卷积层添加到网络中。

In [8]:
# Add a Conv3D layer to your network with relu activation
down_depth_1_layer_0 = Conv3D(filters=64, 
                kernel_size=(3,3,3),
                padding='same',
                strides=(1,1,1)
               )(down_depth_0_layer_pool)
down_depth_1_layer_0 = Activation('relu')(down_depth_1_layer_0)
down_depth_1_layer_0

<KerasTensor: shape=(None, 64, 80, 80, 8) dtype=float32 (created by layer 'activation_2')>

#### 2.4.2 Depth 1,  Layer 1

对于 depth 1 的 layer 1 用于计算filters数量的公式为:
$$filters_{i} = 32 \times (2^{i}) \times 2$$

其中 $i$ 是当前网络深度。 
- Notice that the '$\times 2$' at the end of this expression isn't there for layer 0.

所以在 depth $i=1$:
$$filters_{0} = 32 \times (2^{1}) \times 2 = 128$$

运行下一个单元格，将带有128个filters的3D卷积层添加到网络中。

In [9]:
# Add another Conv3D with 128 filters to your network.
down_depth_1_layer_1 = Conv3D(filters=128, 
                kernel_size=(3,3,3),
                padding='same',
                strides=(1,1,1)
               )(down_depth_1_layer_0)
down_depth_1_layer_1 = Activation('relu')(down_depth_1_layer_1)
down_depth_1_layer_1

<KerasTensor: shape=(None, 128, 80, 80, 8) dtype=float32 (created by layer 'activation_3')>

#### 在 depth 1 中没有最大池化操作 (U型网络的底部)

当到达U-Net网络的“底部”时，不需要在卷积之后应用最大池化操作。

## 3.  解码器(上行) 

现在，我们将研究U-Net网络的解码器路径（查看结构框图时，从右侧向上）。图像的(height, width, length)都在解码器路径中变大。

### 3.1 Depth 0, Up Sampling Layer 0

我们将使用(2,2,2)的池化尺寸进行上采样。
- 这是 [tf.keras.layers.UpSampling3D] 的默认值(https://www.tensorflow.org/api_docs/python/tf/keras/layers/UpSampling3D)
- 我们使用下采样的最后一层作为depth 1处上采样的输入。此时，它是depth 1 的 layer 1。

运行下一个单元格，将上采样操作添加到网络中。

注意，我们没有向该上采样层添加任何激活层。

In [10]:
# Add an upsampling operation to your network
up_depth_0_layer_0 = UpSampling3D(size=(2,2,2))(down_depth_1_layer_1)
up_depth_0_layer_0

<KerasTensor: shape=(None, 128, 160, 160, 16) dtype=float32 (created by layer 'up_sampling3d')>

### 3.2 连接上采样后的 Depth 0 和 下采样后的 Depth 0

现在将使用两个depth均为0的层进行连接操作。
- up_depth_0_layer_0: shape is (?, 128, 160, 160, 16)
- depth_0_layer_1: shape is (?, 64, 160, 160, 16)

- 再次检查这两层的height, width 和 length是否相同。
- 如果它们相同，则可以沿通道轴进行连接。
- 它们的(height, width, length) 都是 (160, 160, 16)。

运行下一个单元格，检查要连接层的height, width 和 length是否相同。

In [12]:
# Print the shape of layers to concatenate
print(up_depth_0_layer_0.shape)
print()
print(down_depth_0_layer_1.shape)

(None, 128, 160, 160, 16)

(None, 64, 160, 160, 16)


运行下一个单元格，将连接操作添加到网络中。

In [13]:
# Add a concatenation along axis 1
up_depth_1_concat = concatenate([up_depth_0_layer_0,
                                 down_depth_0_layer_1],
                                axis=1)
up_depth_1_concat

<KerasTensor: shape=(None, 192, 160, 160, 16) dtype=float32 (created by layer 'concatenate')>

注意，上采样层有128个通道，下行卷积层有64个通道，因此当连接时，结果有128+64=192个通道。

### 3.3 上行卷积 Layer 1

该层filters的数量将设置为相同 depth 0 中下行卷积 layer 1（down_depth_0_layer_1）的信道数量。

运行下一个单元格，以查看 depth 0 layer 1 下行卷积的形状。

In [16]:
down_depth_0_layer_1

<KerasTensor: shape=(None, 64, 160, 160, 16) dtype=float32 (created by layer 'activation_1')>

Notice the number of channels for `depth_0_layer_1` is 64

In [14]:
print(f"number of filters: {down_depth_0_layer_1.shape[1]}")

number of filters: 64


In [17]:
# Add a Conv3D up-convolution with 64 filters to your network
up_depth_1_layer_1 = Conv3D(filters=64, 
                            kernel_size=(3,3,3),
                            padding='same',
                            strides=(1,1,1)
                           )(up_depth_1_concat)
up_depth_1_layer_1 = Activation('relu')(up_depth_1_layer_1)
up_depth_1_layer_1

<KerasTensor: shape=(None, 64, 160, 160, 16) dtype=float32 (created by layer 'activation_5')>

### 3.4 上行卷积 Depth 0, Layer 2

在上行卷积 depth 为 0 的 layer 2中，下一步将添加另一个上行卷积。后面一个上行卷积的filters数量等于下行卷积 depth 0 layer 1 中的filters数量。

运行下一个单元格，以查看下行卷积 depth 0 layer 1 中的filters数量。

前面我们已经知道，'down_depth_0_layer_1' 中的通道数量为64。

运行下一个单元格，将带有64个filters的3D上行卷积添加到网络中。

In [18]:
# Add a Conv3D up-convolution with 64 filters to your network
up_depth_1_layer_2 = Conv3D(filters=64, 
                            kernel_size=(3,3,3),
                            padding='same',
                            strides=(1,1,1)
                           )(up_depth_1_layer_1)
up_depth_1_layer_2 = Activation('relu')(up_depth_1_layer_2)
up_depth_1_layer_2

<KerasTensor: shape=(None, 64, 160, 160, 16) dtype=float32 (created by layer 'activation_6')>

## 4. 最终卷积层

对于最终卷积层，filters的数量设置为等于输入数据中的类别数量。

在本实践中，数据总共有3个类别。

运行下一个单元格，向网络中添加带有3个filters的最终3D卷积。

In [19]:
# Add a final Conv3D with 3 filters to your network.
final_conv = Conv3D(filters=3, #3 categories 
                    kernel_size=(1,1,1),
                    padding='valid',
                    strides=(1,1,1)
                    )(up_depth_1_layer_2)
final_conv

<KerasTensor: shape=(None, 3, 160, 160, 16) dtype=float32 (created by layer 'conv3d_8')>

### 4.1 最终卷积层的激活层

运行下一个单元格，向最终卷积层中添加sigmoid激活层积。

In [20]:
# Add a sigmoid activation to your final convolution.
final_activation = Activation('sigmoid')(final_conv)
final_activation

<KerasTensor: shape=(None, 3, 160, 160, 16) dtype=float32 (created by layer 'activation_7')>

### 4.2 创建并编译模型

在本例中，我们将损失函数和评价指标设置为Keras中的预设选项。然而，在任务中，我们也可以实施更好的损失函数和指标来评估模型性能。

Run the next cell to define and compile your model based on the architecture you created above.
运行下一个单元格，以创建和编译我们上面定义的模型。

In [21]:
# Define and compile your model
model = Model(inputs=input_layer, outputs=final_activation)
model.compile(optimizer=Adam(learning_rate=0.00001),
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy']
             )

In [22]:
# Print out a summary of the model you created
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 4, 160, 160  0           []                               
                                , 16)]                                                            
                                                                                                  
 conv3d (Conv3D)                (None, 32, 160, 160  3488        ['input_1[0][0]']                
                                , 16)                                                             
                                                                                                  
 activation (Activation)        (None, 32, 160, 160  0           ['conv3d[0][0]']                 
                                , 16)                                                         

#### 祝贺你！您已经创建了自己的U-Net模型架构！
接下来，通过将模型摘要与下面定义的示例模型进行比较，检查我们是否正确完成了所有工作。
#### 4.2.1仔细检查我们的模型
要仔细检查我们是否创建了正确的模型，使用我们提供的函数来创建相同的模型，并检查网络层和其尺寸是否匹配！

In [23]:
# Import predefined utilities
import util

In [24]:
# Create a model using a predefined function
model_2 = util.unet_model_3d(depth=2,
                                loss_function='categorical_crossentropy',
                                metrics=['categorical_accuracy'])

In [33]:
# Print out a summary of the model created by the predefined function
model_2.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 4, 160, 160  0           []                               
                                , 16)]                                                            
                                                                                                  
 conv3d_18 (Conv3D)             (None, 32, 160, 160  3488        ['input_3[0][0]']                
                                , 16)                                                             
                                                                                                  
 activation_17 (Activation)     (None, 32, 160, 160  0           ['conv3d_18[0][0]']              
                                , 16)                                                       

#### 查看我们创建的U-Net模型摘要，并将其与上面导入的预定义函数创建的示例模型摘要进行比较。

#### 这就是本此实践的全部内容，我们希望这能让你对本次课程中所介绍的网络体系结构有更多的了解！